In [ ]:
import ipywidgets as widgets
from IPython.display import display
import tempfile

# Create file picker widget accepting PDFs and additional document types.
upload_widget = widgets.FileUpload(
    accept=".pdf,.epub,.mobi,.docx,.txt",  # File types to accept
    multiple=False  # Only one file at a time
)

def handle_upload(change):
    for filename, file_info in upload_widget.value.items():
        content = file_info["content"]
        # Use tempfile to store the uploaded file temporarily
        ext = '.' + filename.split('.')[-1] if '.' in filename else ''
        with tempfile.NamedTemporaryFile(delete=False, prefix="uploaded_", suffix=ext) as temp_file:
            temp_file.write(content)
            print(f"File '{filename}' saved to: {temp_file.name}")

# Observe changes in the widget value; when a file is uploaded, handle it.
upload_widget.observe(handle_upload, names="value")
display(upload_widget)




FileUpload(value=(), accept='.pdf,.epub,.mobi,.docx,.txt', description='Upload')

No file uploaded yet.


In [17]:
import fitz  # PyMuPDF

def extract_pdf_content(file_path):
    doc = fitz.open(file_path)
    content = ""
    structural_elements = []  # list to store detected headings as (text, page number, font size, font name)
    
    # Process each page in the document
    for page_idx in range(len(doc)):
        page = doc[page_idx]
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:
            if b.get("type") == 0:  # this is a text block
                for line in b["lines"]:
                    for span in line["spans"]:
                        text = span["text"]
                        fontsize = span["size"]
                        fontname = span["font"]
                        content += text + " "
                        
                        # Using a naive heuristic to detect headings/chapter titles:
                        # If the font size is larger than 14 or the font indicates bold style
                        if fontsize > 14 or "Bold" in fontname or "bold" in fontname:
                            structural_elements.append((text, page_idx + 1, fontsize, fontname))
                    content += "\n"
    return content, structural_elements

# Example usage:
# Process the file paths stored in uploaded_files if available.
if uploaded_files:
    for filename, file_path in uploaded_files.items():
        if filename.lower().endswith(".pdf"):
            text, headings = extract_pdf_content(file_path)
            print("Extracted text:")
            print(text)
            print("\nInferred Headings/Chapters:")
            for h in headings:
                print("Heading:", h[0], "| Page:", h[1], "| Font Size:", h[2], "| Font:", h[3])
else:
    print("No uploaded files to process.")


No uploaded files to process.
